# Lab 3: Script-kiddie Machine Learning Model Fitting and Evaluation

### Objectives

* Task uses one dataset, credit card fraud. load using fetch_openml
* Use sklearn pipeline from that one example for test-train-split, preprocessing and fitting
* Fit 3 different classifiers models (e.g., logistic regression, SVM, decision tree) with their defaults
* Plot the three models on a ROC curve and on a precision-recall curve
* For each model, report:
    * accuracy
    * AUC of ROC
    * AUC of precision-recall
* For the model with the highest precision-recall AUC, report the cutoff threshold that maximizes the F1 score. Report the confusion matrix for the same threshold.

In [2]:
# Setup/Imports
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml

In [2]:
# Load credit card fraud dataset using fetch_openml
# Note: scikit-learn NEEDS TO BE UPDATED to use as_frame parameter
# > conda list scikit-learn
# > pip install -U scikit-learn

X, y = fetch_openml(data_id='42175', return_X_y=True, as_frame=True) 

print('n records: {}'.format(len(X.index)))

n records: 284807
